In [3]:
from bs4 import BeautifulSoup
import requests
import math
from dateutil import parser
import html
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException,NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
import random
from datetime import datetime
import re
import time

In [70]:
def scrapCorpus(sources,keyword,nb_docs):

    source = ""
    position = ""
    company = ""
    workplace = ""
    published_date = ""
    contract_type = ""
    long_infos = ""


    if  "emploi-territorial" in sources:
        pages = math.ceil(nb_docs/20)
        source = "emploi-territorial"
        rootLink = "https://www.emploi-territorial.fr"
        # 20 offres par pages
        # pages = math.ceil(nb_docs/20)
        url = f"{rootLink}/emploi-mobilite/?adv-search={keyword}&page={pages}"
        response = requests.get(url) 
        soup = BeautifulSoup(response.content, 'html.parser')
        root = soup.find("body")
        offresLinkElems = root.select("div[class*='bloc-lien-offre'] > a[class*='lien-details-offre']")[:nb_docs]    
        links = [rootLink+offresLinkElem.get("href") for offresLinkElem in offresLinkElems]

        corpus = list(dict())

        for link in links:
            response = requests.get(link)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            root = soup.find("body")

            position = root.select("h2[class*='set-line-emploi']")[0].text.strip()
            company = root.select("div[class*='offre-item-value'] > strong > a")[0].text.strip() if root.select("div[class*='offre-item-value'] > strong > a") else "NULL"
            workplace = root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value").text.strip() if root.select_one("div[class*='offre-item-label']:contains('Lieu de travail') + .offre-item-value") else "NULL"
            published_date = root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green").text.strip() if root.select_one("div[class*='px-3']:contains('Publiée le') > .set-color-green") else "NULL"
            contract_type = root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value').text.strip() if root.select_one('div[class*="offre-item-label"]:contains("Type d\'emploi") + .offre-item-value') else "NULL"
            position_type = root.select_one('div[class*="offre-item-label"]:contains("Famille de métiers") + .offre-item-value').text.split(">")[0].strip()
            
            long_text = root.select('div[class*="offre-item-text"]')

            long_infos = " ".join([long_text[0].text.strip(),long_text[1].text.strip(),long_text[2].text.strip()])
            current_offer = {"source": source, "link": rootLink, "position": position, "position_type": position_type,
                "company": company, "workplace": workplace, "published_date": published_date,
                "contract_type": contract_type, "description": long_infos}

            # Check if the offer is already in the corpus based on a frozenset comparison
            corpus.append(current_offer)
    
    if "apec" in sources:
        pages = math.ceil(nb_docs/20)
        source = "apec"
        service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service)
        rootLink = "https://www.apec.fr"
        try:
            for page in range(pages):    
                try:
                    driver.get(f'{rootLink}/candidat/recherche-emploi.html/emploi?motsCles={keyword}&page={pages}')
                    # Initial find of elements
                    try:
                        deny_cookies_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-reject-all-handler'))
                        )

                        deny_cookies_button.click()
                    except TimeoutException:
                        pass

                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                    )

                    # Getting docs left if last page to query
                    if nb_docs%20 != 0 and page==pages-1: 
                        limit = nb_docs%20
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')[:limit]
                    else :
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')
                        
                    for index in range(len(div_elements_to_click_list)):
                        try:
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Re-find elements after navigating back
                            div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, 'div.card-offer')

                            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div.card-offer"))
                            )
                            # Check if the index is within the valid range

                            div_elements_to_click = div_elements_to_click_list[index]
                            company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                            contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                            workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                            published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                            # Scroll into view

                            actions = ActionChains(driver)
                            actions.move_to_element(div_elements_to_click).click().perform()

                            # Wait for the child element to become present in the DOM
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_element_located((By.XPATH, f"//h4[text()='Descriptif du poste']"))
                            )
                            # Get the page source after the click
                            page_source = driver.page_source

                            # Use Beautiful Soup to parse the page source
                            soup = BeautifulSoup(page_source, 'html.parser')

                            # Example: Retrieve the text of a specific element
                            desc = soup.select("h4:contains('Descriptif du poste') + p")
                            descText = ["".join(elem.text) for elem in desc][0]

                            profile = soup.select("h4:contains('Profil recherché') + p")
                            profileText = ["".join(elem.text) for elem in profile][0]                  

                            position = soup.select_one("h4:contains('Métier') + span").text
                            position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
                            long_infos = " ".join([descText,profileText])
                            
                            corpus.append({"source":source,"link":rootLink,"position":position,"position_type":position_type,"company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
                        except Exception as e:
                            # Print the exception for debugging purposes
                            print(f"Error: {e}")

                        finally:
                            # Navigate back to the main page
                            driver.execute_script("window.history.go(-1);")

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")

        finally:
            driver.quit()
    if "hellowork" in sources:
        pages = math.ceil(nb_docs/30)
        source = "hellowork"
        rootLink = "https://www.hellowork.com"
        service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service) 
        try:
            for page in range(pages):    
                try:
                    print(page)
                    driver.get(f'{rootLink}/fr-fr/emploi/recherche.html?k={keyword}&k_autocomplete=&l=France&l_autocomplete=&p={page+1}')
                    # Initial find of elements
                    try:
                        accept_cookies_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#hw-cc-notice-accept-btn'))
                        )

                        accept_cookies_button.click()

                        combobox = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "select[name='country']"))
                        )

                        select = Select(combobox)
                        # Select a specific item by visible text
                        select.select_by_value("FR")

                        form = driver.find_element(By.CSS_SELECTOR,"form[data-action*='service-adaptation']")
                        next_button = form.find_element(By.CSS_SELECTOR,"button")
                        next_button.click()
                    except TimeoutException:
                        pass

                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "button[data-cy='seeOffer']"))
                    )
                    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    # Getting docs left if last page to query
                    if nb_docs%20 != 0 and page==pages-1: 
                        limit = nb_docs%20
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, "button[data-cy='seeOffer']")[:limit]
                    else :
                        div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, "button[data-cy='seeOffer']")
                        
                    for index in range(len(div_elements_to_click_list)):
                        try:
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "button[data-cy='seeOffer']"))
                            )

                            # Re-find elements after navigating back
                            div_elements_to_click_list = driver.find_elements(By.CSS_SELECTOR, "button[data-cy='seeOffer']")
                            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                            WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "button[data-cy='seeOffer']"))
                            )
                            # Check if the index is within the valid range

                            div_elements_to_click = div_elements_to_click_list[index]
                            # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
                            # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                            # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                            # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                            # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                            # Scroll into view

                            # actions = ActionChains(driver)
                            # actions.move_to_element(div_elements_to_click).click().perform()
                            #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
                            div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
                            company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                            company = company_elem.find_element(By.CSS_SELECTOR, "span").text
                            company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                            contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
                            workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
                            published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

                            driver.execute_script("arguments[0].click();", div_elements_to_click)

                            # Wait for the child element to become present in the DOM
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "span[data-cy='jobTitle']"))
                            )
                            # Get the page source after the click
                            page_source = driver.page_source

                            # Use Beautiful Soup to parse the page source
                            soup = BeautifulSoup(page_source, 'html.parser')

                            # Example: Retrieve the text of a specific element
                            # desc = soup.select("h4:contains('Descriptif du poste') + p")
                            # descText = ["".join(elem.text) for elem in desc][0]

                            # profile = soup.select("h4:contains('Profil recherché') + p")
                            # profileText = ["".join(elem.text) for elem in profile][0]                  

                            position = soup.select_one("span[data-cy='jobTitle']").text
                            # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
                            long_infos = soup.select("section")
                            infos = [item.text.replace("\n"," ").strip() for item in long_infos]
                            infos = [re.sub(r'\s+', ' ',item) for item in infos]
                            infos = " ".join([item for item in infos if item != ''])
                            # infos = [item for item in infos if '' not in item]
                            corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":infos})
                        except Exception as e:
                            # Print the exception for debugging purposes
                            print(f"Error: {e}")

                        finally:
                            # Navigate back to the main page
                            driver.execute_script("window.history.go(-1);")

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")
        finally:
            driver.quit()
 
    if "welcometothejungle" in sources:
        source = "welcometothejungle"
        rootLink = "https://www.welcometothejungle.com"
        service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
        driver = webdriver.Chrome(service=service)
        try:
            for page in range(1,pages+1):    
                try:
                    driver.get(f'{rootLink}/fr/jobs?query=data&aroundQuery=France&page={page}')
                    # Initial find of elements
                    try:
                        accept_cookies_button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#axeptio_btn_acceptAll'))
                        )

                        accept_cookies_button.click()
                    except TimeoutException:
                        pass
                        
                    offers = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "ol[data-testid='search-results']"))
                    )
                    
                    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    # Getting docs left if last page to query
                    if nb_docs%30 != 0 and page==pages: 
                        limit = nb_docs%30
                        div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")[:limit]
                    else :
                        div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")

                    for index in range(len(div_elements_to_click_list)):
                        try:
                            offers = WebDriverWait(driver, 10).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "ol[data-testid='search-results']"))
                            )
                            
                            div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")
                            div_elements_to_click = div_elements_to_click_list[index]
                            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                            # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                            # Check if the index is within the valid range
                            div_offer = div_elements_to_click.find_element(By.CSS_SELECTOR,"div")

                            position_elem = div_offer.find_element(By.CSS_SELECTOR,"h4")
                            position = position_elem.text
                            workplace = position_elem.find_elements(By.XPATH, "./following::*//span")[1].text
                            contract_type_icon = div_offer.find_element(By.CSS_SELECTOR,"i[name='contract']")
                            main_contract_div = driver.execute_script("return arguments[0].parentNode;", contract_type_icon)
                            contract_type = main_contract_div.find_element(By.CSS_SELECTOR,"span").text
                            published_date = div_offer.find_element(By.CSS_SELECTOR,"time").get_attribute("datetime").split("T")[0]
                            published_date = datetime.strptime(published_date, '%Y-%m-%d')
                            published_date = published_date.strftime('%d/%m/%Y')
                            # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
                            # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                            # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                            # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                            # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                            # Scroll into view
                            # actions = ActionChains(driver)
                            # actions.move_to_element(div_elements_to_click).click().perform()
                            #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
                            # div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
                            # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                            # company = company_elem.find_element(By.CSS_SELECTOR, "span").text
                            # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                            # contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
                            # workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
                            # published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

                            driver.execute_script("arguments[0].click();", div_offer)
                            WebDriverWait(driver, 20).until(
                                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='job-section-description']"))
                            )
                            description_elem = driver.find_element(By.CSS_SELECTOR,"div[data-testid='job-section-description']")

                            # driver.execute_script("arguments[0].click();",description_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))
                            
                            WebDriverWait(driver, 3)

                            try:
                                profile_elem = driver.find_element(By.CSS_SELECTOR, "div[data-testid='job-section-experience']")
                                profile_html = profile_elem.find_elements(By.CSS_SELECTOR, "div")[-2].get_attribute("innerHTML")
                                profile = BeautifulSoup(profile_html, 'html.parser').get_text()
                            except NoSuchElementException:
                                # If profile element is not found, set it to "NULL"
                                profile = "NULL"
                            # driver.execute_script("arguments[0].click();",profile_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))

                            description = description_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                            # profile = profile_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                            
                            #parsing tags
                            description = BeautifulSoup(description, 'html.parser').get_text()
                            company_elem = driver.find_element(By.CSS_SELECTOR,"a[href*='/fr/companies/']")
                            company = company_elem.find_element(By.CSS_SELECTOR,"img").get_attribute("alt")
                            long_infos = " ".join([description,profile])
                            # print(long_infos)
                            # Wait for the child element to become present in the DOM

                            # # Get the page source after the click
                            # page_source = driver.page_source

                            # Use Beautiful Soup to parse the page source
                            # soup = BeautifulSoup(page_source, 'html.parser')

                            # Example: Retrieve the text of a specific element
                            # desc = soup.select("h4:contains('Descriptif du poste') + p")
                            # descText = ["".join(elem.text) for elem in desc][0]

                            # profile = soup.select("h4:contains('Profil recherché') + p")
                            # profileText = ["".join(elem.text) for elem in profile][0]                  

                            # position = soup.select_one("span[data-cy='jobTitle']").text
                            # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
                            # long_infos = soup.select("section")
                            # infos = [item.text.replace("\n"," ").strip() for item in long_infos]
                            # infos = [re.sub(r'\s+', ' ',item) for item in infos]
                            # infos = " ".join([item for item in infos if item != ''])
                            # infos = [item for item in infos if '' not in item]
                            corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
                        except Exception as e:
                            # Print the exception for debugging purposes
                            print(f"Error: {e}")

                        finally:
                            # Navigate back to the main page
                            driver.execute_script("window.history.go(-1);")

                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")
        finally:
            driver.quit()

    return(corpus)


In [71]:
corpus = scrapCorpus(sources=["emploi-territorial","hellowork","welcometothejungle"],keyword="data",nb_docs=10)

0


[{'source': 'hellowork',
  'link': 'https://www.hellowork.com',
  'position': 'Data Officer - Data Factory H/F',
  'position_type': 'NULL',
  'company': 'EURO-INFORMATION DEVELOPPEMENTS',
  'workplace': 'Nantes - 44',
  'published_date': '27/12/2023',
  'contract_type': 'CDI',
  'description': "Les missions du poste Notre raison d'être : Ensemble, Ecouter et Agir.Vos missionsVous souhaitez travailler dans un environnement innovant et au sein d'une équipe à taille humaine ? La Data Factory est le département dédié à la donnée au sein d'Euro-Information au service de l'ensemble des entités du groupe et la diversité de leurs métiers pour la valorisation des données.- Nous fournissons les environnements (Vertica, Hadoop) et la technologie (Jupyter Lab pour python, R, SPSS, SAS, SAP BO ou IB Webfocus).- Nous les peuplons de données (plus de 90 billions de données et en extension).- Nous en assurons la sécurité et le respect de l'intimité numérique des clients.- Nous accompagnons les métiers

In [77]:
[item for item in corpus if item["source"]=="emploi-territorial"]

[{'source': 'emploi-territorial',
  'link': 'https://www.emploi-territorial.fr',
  'position': 'Ingénieur data F/H',
  'position_type': "Informatique et système d'information",
  'company': 'CONSEIL DEPARTEMENTAL DU NORD',
  'workplace': 'Lille cedex',
  'published_date': '19/12/2023',
  'contract_type': "Emploi permanent - création d'emploi",
  'description': "Le Département du Nord place les systèmes d'information au cœur du projet de transformation digitale au service de l'usager et de ses partenaires. La Direction des Systèmes d'Information (environ 110 collaborateurs, 7000 postes de travail, 200 sites) poursuit le développement de projets innovants :\n-\tLa sécurisation de ses infrastructures, le développement du haut débit pour ses établissements, des plateformes téléphoniques, des systèmes de GED et de numérisation.\n-\tDe nombreux chantiers autour du poste de travail comme la mobilité en autres\n-\tLa mise en œuvre d'applications au service des compétences départementales, la s

In [11]:
corpus

[{'source': 'hellowork',
  'link': 'https://www.hellowork.com',
  'position': 'Data Modeler H/F',
  'position_type': 'NULL',
  'company': 'VOLT',
  'workplace': 'Alpes-Maritimes - 06',
  'published_date': '22/12/2023',
  'contract_type': 'CDI',
  'description': "Les missions du poste Nous recherchons pour l'un de nos clients un Data modeler. Le poste est à pourvoir aux alentours de Nice.Vos missions :- Soutien renforcé sur la gestion des données- Modélisation du Data Warehouse (DWH)- Collaboration au sein de l'équipe de data engineering- Travaux axés sur Spotfire pour la Business Intelligence (BI)Votre environnement technique :- Data warehouses et la modélisation dimensionnelle (en étoile)- Méthode spécifique de modélisation et de conception de systèmes décisionnels Kimball- Avoir une bonne maîtrise du langage SQL- Idéalement, avoir des compétences en Sas Designer, Python et Excel- Méthodes Agiles notamment Kanban/Scrum Le profil recherché - Vous justifiez de minimum de 4 ans d'expérie

In [130]:
from sqlalchemy import create_engine, Column, Integer, Date, ForeignKey,text,String
from sqlalchemy.orm import declarative_base, Session, relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import IntegrityError

Base = declarative_base()

# Modele de bdd
class HPositionType(Base):
    __tablename__ = 'h_position_type'
    id = Column(Integer, primary_key=True)
    position_type = Column(String)
    d_positions = relationship('DPosition', back_populates='h_position_type')

class DPosition(Base):
    __tablename__ = 'd_position'
    id = Column(Integer, primary_key=True)
    position = Column(String)
    position_type_id = Column(Integer, ForeignKey('h_position_type.id'))
    h_position_type = relationship('HPositionType', back_populates='d_positions')

class DWebsite(Base):
    __tablename__ = 'd_website'
    id = Column(Integer, primary_key=True)
    label = Column(String)
    link = Column(String)

class DCompany(Base):
    __tablename__ = 'd_company'
    id = Column(Integer, primary_key=True)
    label = Column(String)

class DCity(Base):
    __tablename__ = 'd_city'
    id = Column(Integer, primary_key=True)
    city = Column(String)

class DContractType(Base):
    __tablename__ = 'd_contract_type'
    id = Column(Integer, primary_key=True)
    contract_type = Column(String)

class DCalendar(Base):
    __tablename__ = 'd_calendar'
    date = Column(Date, primary_key=True)
    day = Column(Integer)
    month = Column(Integer)
    year = Column(Integer)


class FJobAdvertisements(Base):
    __tablename__ = 'f_job_advertisements'
    nb_occurences = Column(Integer)
    contract_type_id = Column(Integer, ForeignKey('d_contract_type.id'),primary_key=True)
    position_id = Column(Integer, ForeignKey('d_position.id'),primary_key=True)
    website_id = Column(Integer, ForeignKey('d_website.id'),primary_key=True)
    city_id = Column(Integer, ForeignKey('d_city.id'),primary_key=True)
    company_id = Column(Integer, ForeignKey('d_company.id'),primary_key=True)
    published_date = Column(Date, ForeignKey('d_calendar.date'),primary_key=True)

    position = relationship('DPosition', back_populates='job_advertisements')
    website = relationship('DWebsite', back_populates='job_advertisements')
    company = relationship('DCompany', back_populates='job_advertisements')
    city = relationship('DCity', back_populates='job_advertisements')
    contract_type = relationship('DContractType', back_populates='job_advertisements')

# Définition des relations
DPosition.job_advertisements = relationship('FJobAdvertisements', back_populates='position')
DWebsite.job_advertisements = relationship('FJobAdvertisements', back_populates='website')
DCompany.job_advertisements = relationship('FJobAdvertisements', back_populates='company')
DCity.job_advertisements = relationship('FJobAdvertisements', back_populates='city')
DContractType.job_advertisements = relationship('FJobAdvertisements', back_populates='contract_type')


In [30]:
from datetime import datetime, timedelta
#Seulement à executer une fois (contrainte avec la table de fait)
def insertCalendar(yearsRange):
    engine = create_engine('mysql+mysqlconnector://root:@localhost/job_scrapping')
    Base.metadata.create_all(engine)
    session = Session(bind=engine)
    session.commit()
    for year in yearsRange:
        for month in range(1, 13):
            num_days_in_month = (datetime(year, month % 12 + 1, 1) - timedelta(days=1)).day
            for day in range(1, num_days_in_month + 1):
                date_details = datetime(year, month, day)
                date = DCalendar(date=date_details.strftime('%Y-%m-%d'),day=date_details.day,month=date_details.month,year=date_details.year)
                session.add(date)
                session.commit()
    session.close()

In [ ]:
insertCalendar(range(2015,2025))

In [160]:
#Alimentation du DW
def fillDW(corpus):
    engine = create_engine('mysql+mysqlconnector://root:@localhost/job_scrapping')
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    duplicates = False
    #Vidage du DW
    session = Session()
    session.execute(text("CALL pTruncateDW()"))
    session.commit()
    i=0
    #Itération sur les éléments du corpus récupérés
    for item in corpus:
        corpus_position = item["position"]
        corpus_website = item["source"]
        print(corpus_position,corpus_website)
        corpus_link = item["link"]
        corpus_company = item["company"]
        corpus_city = item["workplace"]
        corpus_contract_type = item["contract_type"]
        corpus_position_type = item['position_type']
        corpus_published_date = item['published_date']
        try:
                date_object = datetime.strptime(corpus_published_date, '%d/%m/%Y')
                # If successful, use the formatted date
                corpus_published_date = date_object.strftime('%Y-%m-%d')
        except ValueError:
            corpus_published_date = datetime.now().strftime('%Y-%m-%d')
        # print(corpus_position)
        # print(corpus_position_type)
        
        position_type = HPositionType(position_type=corpus_position_type)
        position = DPosition(position=corpus_position)
        website = DWebsite(label=corpus_website,link=corpus_link)
        company = DCompany(label=corpus_company)
        city = DCity(city=corpus_city)
        contract_type = DContractType(contract_type=corpus_contract_type)
        session = Session()

        # Recherche des élements de chaque dimensions pour éviter les doublons
        existing_position_type = session.query(HPositionType).filter_by(position_type=corpus_position_type).first()
        existing_position = session.query(DPosition).filter_by(position=corpus_position).first()
        existing_website = session.query(DWebsite).filter_by(label=corpus_website).first()
        existing_company = session.query(DCompany).filter_by(label=corpus_company).first()
        existing_city = session.query(DCity).filter_by(city=corpus_city).first()
        existing_contract_type = session.query(DContractType).filter_by(contract_type=corpus_contract_type).first()

        #Si elle existe je récupère la ligne existante, sinon j'insert la nouvelle ligne
        if existing_position_type:
            position_type = existing_position_type
        else:
            session.add(position_type)
            session.commit()
        if existing_position:
            position = existing_position
        else:
            position.position_type_id = position_type.id
            session.add(position)

        if existing_website:
            website = existing_website
        else:
            session.add(website)

        if existing_company:
            company = existing_company
        else:
            session.add(company)

        if existing_city:
            city = existing_city
        else:
            session.add(city)

        if existing_contract_type:
            contract_type = existing_contract_type
        else:
            session.add(contract_type)
        session.commit()
        
        # J'insert les données dans la table de fait (id des dimensions + KPI)
        job_advertisement = FJobAdvertisements(
            nb_occurences=random.randint(3,10),
            position=position,
            website=website,
            company=company,
            city=city,
            contract_type=contract_type,
            published_date = corpus_published_date
        )

        session.add(job_advertisement)
        try:
            session.add(job_advertisement)
            session.commit()
            i = i + 1
        except IntegrityError as e:
            duplicates = True
            session.rollback()
        session.commit()
    session.close()
    if duplicates:
        print(f"Some exact duplicates have been detected in the job scrapping process, less documents have been saved then asked : {i} documents saved")

In [161]:
fillDW(corpus)

Chef de projet Data (H/F) welcometothejungle
Data Project Coordinator / Chargé(e) de données PIM DAM (F/H) welcometothejungle
Data Analytics Engineer H/F/X welcometothejungle
Manageur (euse) Data et Analytics welcometothejungle
Data Center Security Manager, DC Security welcometothejungle
Senior Data Scientist welcometothejungle
Consultant.e Data Marketing Confirmé.e welcometothejungle
Data Analyst confirmé - (F/H) welcometothejungle
Scrum Master Data F/H welcometothejungle
Data Privacy Analyst F/H welcometothejungle
DATA ANALYST EXPERT POWER BI welcometothejungle
Data Engineer - Expert Big Data H/F welcometothejungle
Auditeur expérimenté Outils, Data & Méthodologie F/H welcometothejungle
Ingénieur Big Data - Banque - Bordeaux welcometothejungle
Consultant junior Data x Business Consulting H/F - Stage de fin d'études welcometothejungle
Data Analyst (Stage de fin d'étude) welcometothejungle
Product Owner Data Marketing H/F/X welcometothejungle
Data engineer F/H welcometothejungle
Ingénie

In [158]:
source = "welcometothejungle"
rootLink = "https://www.welcometothejungle.com"
service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service) 
keyword="data"
pages=2
nb_docs=33
corpus = list()
try:
    for page in range(1,pages+1):    
        try:
            driver.get(f'{rootLink}/fr/jobs?query=data&aroundQuery=France&page={page}')
            # Initial find of elements
            try:
                accept_cookies_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#axeptio_btn_acceptAll'))
                )

                accept_cookies_button.click()
            except TimeoutException:
                pass
                
            offers = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ol[data-testid='search-results']"))
            )
            
            # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Getting docs left if last page to query
            if nb_docs%30 != 0 and page==pages: 
                limit = nb_docs%30
                div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")[:limit]
            else :
                div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")

            for index in range(len(div_elements_to_click_list)):
                try:
                    offers = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "ol[data-testid='search-results']"))
                    )
                    
                    div_elements_to_click_list = offers.find_elements(By.CSS_SELECTOR, "li")
                    div_elements_to_click = div_elements_to_click_list[index]
                    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
                    # Check if the index is within the valid range
                    div_offer = div_elements_to_click.find_element(By.CSS_SELECTOR,"div")

                    position_elem = div_offer.find_element(By.CSS_SELECTOR,"h4")
                    position = position_elem.text
                    workplace = position_elem.find_elements(By.XPATH, "./following::*//span")[1].text
                    contract_type_icon = div_offer.find_element(By.CSS_SELECTOR,"i[name='contract']")
                    main_contract_div = driver.execute_script("return arguments[0].parentNode;", contract_type_icon)
                    contract_type = main_contract_div.find_element(By.CSS_SELECTOR,"span").text
                    published_date = div_offer.find_element(By.CSS_SELECTOR,"time").get_attribute("datetime").split("T")[0]
                    published_date = datetime.strptime(published_date, '%Y-%m-%d')
                    published_date = published_date.strftime('%d/%m/%Y')
                    # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
                    # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
                    # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
                    # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
                    # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
                    # Scroll into view
                    # actions = ActionChains(driver)
                    # actions.move_to_element(div_elements_to_click).click().perform()
                    #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
                    # div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
                    # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                    # company = company_elem.find_element(By.CSS_SELECTOR, "span").text
                    # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
                    # contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
                    # workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
                    # published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

                    driver.execute_script("arguments[0].click();", div_offer)
                    WebDriverWait(driver, 20).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='job-section-description']"))
                    )
                    description_elem = driver.find_element(By.CSS_SELECTOR,"div[data-testid='job-section-description']")

                    # driver.execute_script("arguments[0].click();",description_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))
                    
                    WebDriverWait(driver, 3)

                    try:
                        profile_elem = driver.find_element(By.CSS_SELECTOR, "div[data-testid='job-section-experience']")
                        profile_html = profile_elem.find_elements(By.CSS_SELECTOR, "div")[-2].get_attribute("innerHTML")
                        profile = BeautifulSoup(profile_html, 'html.parser').get_text()
                    except NoSuchElementException:
                        # If profile element is not found, set it to "NULL"
                        profile = "NULL"
                    # driver.execute_script("arguments[0].click();",profile_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))

                    description = description_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                    # profile = profile_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                    
                    #parsing tags
                    description = BeautifulSoup(description, 'html.parser').get_text()
                    company_elem = driver.find_element(By.CSS_SELECTOR,"a[href*='/fr/companies/']")
                    company = company_elem.find_element(By.CSS_SELECTOR,"img").get_attribute("alt")
                    long_infos = " ".join([description,profile])
                    # print(long_infos)
                    # Wait for the child element to become present in the DOM

                    # # Get the page source after the click
                    # page_source = driver.page_source

                    # Use Beautiful Soup to parse the page source
                    # soup = BeautifulSoup(page_source, 'html.parser')

                    # Example: Retrieve the text of a specific element
                    # desc = soup.select("h4:contains('Descriptif du poste') + p")
                    # descText = ["".join(elem.text) for elem in desc][0]

                    # profile = soup.select("h4:contains('Profil recherché') + p")
                    # profileText = ["".join(elem.text) for elem in profile][0]                  

                    # position = soup.select_one("span[data-cy='jobTitle']").text
                    # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
                    # long_infos = soup.select("section")
                    # infos = [item.text.replace("\n"," ").strip() for item in long_infos]
                    # infos = [re.sub(r'\s+', ' ',item) for item in infos]
                    # infos = " ".join([item for item in infos if item != ''])
                    # infos = [item for item in infos if '' not in item]
                    corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
                except Exception as e:
                    # Print the exception for debugging purposes
                    print(f"Error: {e}")

                finally:
                    # Navigate back to the main page
                    driver.execute_script("window.history.go(-1);")

        except Exception as e:
            # Print the exception for debugging purposes
            print(f"Error: {e}")

finally:
    driver.quit()

In [159]:
len(corpus)

33

In [25]:
# # Problème toutes les entreprises sont Adecco, si autre, redirection sur un autre site
# source = "adecco"
# rootLink = "https://www.adecco.fr"
# service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
# driver = webdriver.Chrome(service=service) 
# keyword="data"
# pages=1
# nb_docs=33
# corpus = list()
# try:
#     for page in range(1,pages+1):    
#         try:
#             print(page)
#             driver.get(f'{rootLink}/resultats-offres-emploi/m-{keyword}?pageNum={page}')
#             # Initial find of elements
                
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "a[id='hypViewJob']"))
#             )

#             offers = driver.find_elements(By.CSS_SELECTOR, "a[id='hypViewJob']")

#             # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             # Getting docs left if last page to query
#             if nb_docs%50!= 0 and page==pages: 
#                 limit = nb_docs%50
#                 div_elements_to_click_list = offers[:limit]
#             else :
#                 div_elements_to_click_list = offers
#             # print(len(div_elements_to_click_list))
#             for index in range(len(div_elements_to_click_list)):
#                 try:
#                     WebDriverWait(driver, 10).until(
#                         EC.presence_of_element_located((By.CSS_SELECTOR, "a[id='hypViewJob']"))
#                     )
                    
#                     div_elements_to_click_list = offers
#                     div_elements_to_click = div_elements_to_click_list[index]
#                     # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
#                     # Check if the index is within the valid range
#                     # div_offer = div_elements_to_click.find_element(By.CSS_SELECTOR,"div")

#                     # position_elem = div_offer.find_element(By.CSS_SELECTOR,"h4")
#                     # position = position_elem.text
#                     # workplace = position_elem.find_elements(By.XPATH, "./following::*//span")[1].text
#                     # contract_type_icon = div_offer.find_element(By.CSS_SELECTOR,"i[name='contract']")
#                     # main_contract_div = driver.execute_script("return arguments[0].parentNode;", contract_type_icon)
#                     # contract_type = main_contract_div.find_element(By.CSS_SELECTOR,"span").text
#                     # published_date = div_offer.find_element(By.CSS_SELECTOR,"time").get_attribute("datetime").split("T")[0]
#                     # published_date = datetime.strptime(published_date, '%Y-%m-%d')
#                     # published_date = published_date.strftime('%d/%m/%Y')
#                     # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
#                     # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
#                     # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
#                     # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
#                     # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
#                     # Scroll into view
#                     # actions = ActionChains(driver)
#                     # actions.move_to_element(div_elements_to_click).click().perform()
#                     #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
#                     # div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
#                     # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#                     # company = company_elem.find_element(By.CSS_SELECTOR, "span").text
#                     # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#                     # contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
#                     # workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
#                     # published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

#                     driver.execute_script("arguments[0].click();", div_elements_to_click)

#                     WebDriverWait(driver, 20).until(
#                         EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='media-body']"))
#                     )
                    
#                     workplace = driver.find_element(By.CSS_SELECTOR,"span[id='lblCity']").text
#                     contract_type_elem = driver.find_element(By.CSS_SELECTOR,"span[id='ltEmploymentType']")
#                     contract_type = contract_type_elem.find_element(By.CSS_SELECTOR,"a").text
                    
#                     # driver.execute_script("arguments[0].click();",description_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))
                    
#                     WebDriverWait(driver, 3)

#                     try:
#                         profile_elem = driver.find_element(By.CSS_SELECTOR, "div[data-testid='job-section-experience']")
#                         profile_html = profile_elem.find_elements(By.CSS_SELECTOR, "div")[-2].get_attribute("innerHTML")
#                         profile = BeautifulSoup(profile_html, 'html.parser').get_text()
#                     except NoSuchElementException:
#                         # If profile element is not found, set it to "NULL"
#                         profile = "NULL"
#                     # driver.execute_script("arguments[0].click();",profile_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))

#                     description = description_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
#                     # profile = profile_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                    
#                     #parsing tags
#                     # description = BeautifulSoup(description, 'html.parser').get_text()
#                     # company_elem = driver.find_element(By.CSS_SELECTOR,"a[href*='/fr/companies/']")
#                     # company = company_elem.find_element(By.CSS_SELECTOR,"img").get_attribute("alt")
#                     # long_infos = " ".join([description,profile])
#                     # print(long_infos)
#                     # Wait for the child element to become present in the DOM

#                     # # Get the page source after the click
#                     # page_source = driver.page_source

#                     # Use Beautiful Soup to parse the page source
#                     # soup = BeautifulSoup(page_source, 'html.parser')

#                     # Example: Retrieve the text of a specific element
#                     # desc = soup.select("h4:contains('Descriptif du poste') + p")
#                     # descText = ["".join(elem.text) for elem in desc][0]

#                     # profile = soup.select("h4:contains('Profil recherché') + p")
#                     # profileText = ["".join(elem.text) for elem in profile][0]                  

#                     # position = soup.select_one("span[data-cy='jobTitle']").text
#                     # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
#                     # long_infos = soup.select("section")
#                     # infos = [item.text.replace("\n"," ").strip() for item in long_infos]
#                     # infos = [re.sub(r'\s+', ' ',item) for item in infos]
#                     # infos = " ".join([item for item in infos if item != ''])
#                     # infos = [item for item in infos if '' not in item]
#                     corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
#                 except Exception as e:
#                     # Print the exception for debugging purposes
#                     print(f"Error: {e}")

#                 finally:
#                     # Navigate back to the main page
#                     driver.execute_script("window.history.go(-1);")

#         except Exception as e:
#             # Print the exception for debugging purposes
#             print(f"Error: {e}")

# finally:
#     driver.quit()

1
Intérim
Error: name 'description_elem' is not defined
Error: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.130)
Stacktrace:
	GetHandleVerifier [0x00007FF75A2882B2+55298]
	(No symbol) [0x00007FF75A1F5E02]
	(No symbol) [0x00007FF75A0B05AB]
	(No symbol) [0x00007FF75A0BE59A]
	(No symbol) [0x00007FF75A0B59E9]
	(No symbol) [0x00007FF75A0B5AF3]
	(No symbol) [0x00007FF75A0B4298]
	(No symbol) [0x00007FF75A0B732A]
	(No symbol) [0x00007FF75A12B12B]
	(No symbol) [0x00007FF75A1120AA]
	(No symbol) [0x00007FF75A12AAA4]
	(No symbol) [0x00007FF75A111E83]
	(No symbol) [0x00007FF75A0E670A]
	(No symbol) [0x00007FF75A0E7964]
	GetHandleVerifier [0x00007FF75A600AAB+3694587]
	GetHandleVerifier [0x00007FF75A65728E+4048862]
	GetHandleVerifier [0x00007FF75A64F173+4015811]
	GetHandleVerifier [0x00007FF75A3247D6+695590]
	(No symbol) [0x00007FF75A200CE8]
	(No symbol) [0x00007FF75A1FCF34]
	(No symbol) [0x00007FF75A1FD062]
	(No symbol) [0x00007FF75A1ED3A3]
	BaseThreadI

KeyboardInterrupt: 

In [44]:
# # Problème toutes les entreprises sont Adecco, si autre, redirection sur un autre site
# source = "jobintree"
# rootLink = "https://www.jobintree.com"
# service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
# driver = webdriver.Chrome(service=service) 
# keyword="data"
# pages=1
# nb_docs=33
# corpus = list()
# try:
#     for page in range(pages):    
#         try:
#             driver.get(f'{rootLink}/emploi?keywords={keyword}&page={page}')
#             # Initial find of elements
                
#             offers = WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='annonces_normales']"))
#             )

#             offers = offers.find_elements(By.CSS_SELECTOR,"a")
#             print(len(offers))
#             # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             # Getting docs left if last page to query
#             if nb_docs%20!= 0 and page==pages-1: 
#                 limit = nb_docs%20
#                 div_elements_to_click_list = offers[:limit]
#             else :
#                 div_elements_to_click_list = offers
#             # print(len(div_elements_to_click_list))
#             for index in range(len(div_elements_to_click_list)):
#                 try:
#                     WebDriverWait(driver, 10).until(
#                         EC.presence_of_element_located((By.CSS_SELECTOR, "a[id='hypViewJob']"))
#                     )
                    
#                     div_elements_to_click_list = offers
#                     div_elements_to_click = div_elements_to_click_list[index]
#                     # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#                     # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
#                     # Check if the index is within the valid range
#                     # div_offer = div_elements_to_click.find_element(By.CSS_SELECTOR,"div")

#                     # position_elem = div_offer.find_element(By.CSS_SELECTOR,"h4")
#                     # position = position_elem.text
#                     # workplace = position_elem.find_elements(By.XPATH, "./following::*//span")[1].text
#                     # contract_type_icon = div_offer.find_element(By.CSS_SELECTOR,"i[name='contract']")
#                     # main_contract_div = driver.execute_script("return arguments[0].parentNode;", contract_type_icon)
#                     # contract_type = main_contract_div.find_element(By.CSS_SELECTOR,"span").text
#                     # published_date = div_offer.find_element(By.CSS_SELECTOR,"time").get_attribute("datetime").split("T")[0]
#                     # published_date = datetime.strptime(published_date, '%Y-%m-%d')
#                     # published_date = published_date.strftime('%d/%m/%Y')
#                     # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
#                     # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
#                     # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
#                     # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
#                     # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
#                     # Scroll into view
#                     # actions = ActionChains(driver)
#                     # actions.move_to_element(div_elements_to_click).click().perform()
#                     #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
#                     # div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
#                     # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#                     # company = company_elem.find_element(By.CSS_SELECTOR, "span").text
#                     # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#                     # contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
#                     # workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
#                     # published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

#                     driver.execute_script("arguments[0].click();", div_elements_to_click)

#                     WebDriverWait(driver, 20).until(
#                         EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='media-body']"))
#                     )
                    
#                     workplace = driver.find_element(By.CSS_SELECTOR,"span[id='lblCity']").text
#                     contract_type_elem = driver.find_element(By.CSS_SELECTOR,"span[id='ltEmploymentType']")
#                     contract_type = contract_type_elem.find_element(By.CSS_SELECTOR,"a").text
                    
#                     # driver.execute_script("arguments[0].click();",description_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))
                    
#                     WebDriverWait(driver, 3)

#                     try:
#                         profile_elem = driver.find_element(By.CSS_SELECTOR, "div[data-testid='job-section-experience']")
#                         profile_html = profile_elem.find_elements(By.CSS_SELECTOR, "div")[-2].get_attribute("innerHTML")
#                         profile = BeautifulSoup(profile_html, 'html.parser').get_text()
#                     except NoSuchElementException:
#                         # If profile element is not found, set it to "NULL"
#                         profile = "NULL"
#                     # driver.execute_script("arguments[0].click();",profile_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))

#                     description = description_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
#                     # profile = profile_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
                    
#                     #parsing tags
#                     # description = BeautifulSoup(description, 'html.parser').get_text()
#                     # company_elem = driver.find_element(By.CSS_SELECTOR,"a[href*='/fr/companies/']")
#                     # company = company_elem.find_element(By.CSS_SELECTOR,"img").get_attribute("alt")
#                     # long_infos = " ".join([description,profile])
#                     # print(long_infos)
#                     # Wait for the child element to become present in the DOM

#                     # # Get the page source after the click
#                     # page_source = driver.page_source

#                     # Use Beautiful Soup to parse the page source
#                     # soup = BeautifulSoup(page_source, 'html.parser')

#                     # Example: Retrieve the text of a specific element
#                     # desc = soup.select("h4:contains('Descriptif du poste') + p")
#                     # descText = ["".join(elem.text) for elem in desc][0]

#                     # profile = soup.select("h4:contains('Profil recherché') + p")
#                     # profileText = ["".join(elem.text) for elem in profile][0]                  

#                     # position = soup.select_one("span[data-cy='jobTitle']").text
#                     # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
#                     # long_infos = soup.select("section")
#                     # infos = [item.text.replace("\n"," ").strip() for item in long_infos]
#                     # infos = [re.sub(r'\s+', ' ',item) for item in infos]
#                     # infos = " ".join([item for item in infos if item != ''])
#                     # infos = [item for item in infos if '' not in item]
#                     corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
#                 except Exception as e:
#                     # Print the exception for debugging purposes
#                     print(f"Error: {e}")

#                 finally:
#                     # Navigate back to the main page
#                     driver.execute_script("window.history.go(-1);")

#         except Exception as e:
#             # Print the exception for debugging purposes
#             print(f"Error: {e}")

# finally:
#     driver.quit()

KeyboardInterrupt: 

In [61]:
# source = "pole-emploi"
# rootLink = "https://candidat.pole-emploi.fr"
# service = ChromeService("C:/Users/leogo/Documents/chromedriver-win64/chromedriver-win64/chromedriver.exe")
# driver = webdriver.Chrome(service=service) 
# keyword="data"
# pages=2
# nb_docs=33
# corpus = list()
# try:
#     driver.get(f'{rootLink}/offres/recherche?motsCles={keyword}&range=0-{nb_docs-1}')
#     # Initial find of elements
#     try:
#         accept_cookies_button = WebDriverWait(driver, 10).until(
#             EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#pecookies-accept-all'))
#         )
#         print("test")
#         accept_cookies_button.click()
#     except TimeoutException:
#         pass
#     WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='media with-fav']"))
#     )

#     offers = driver.find_elements(By.CSS_SELECTOR, "a[class='media with-fav']")

#     div_elements_to_click_list = offers
#     # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#     # Getting docs left if last page to query
#     # print(len(div_elements_to_click_list))
#     for index in range(len(div_elements_to_click_list)):
#         try:
#             WebDriverWait(driver, 10).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='media with-fav']"))
#             )
#             offers = driver.find_elements(By.CSS_SELECTOR, "a[class='media with-fav']")

#             div_elements_to_click_list = offers
#             div_elements_to_click = div_elements_to_click_list[index]
#             # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             # driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
#             # Check if the index is within the valid range
#             # div_offer = div_elements_to_click.find_element(By.CSS_SELECTOR,"div")

#             # position_elem = div_offer.find_element(By.CSS_SELECTOR,"h4")
#             # position = position_elem.text
#             # workplace = position_elem.find_elements(By.XPATH, "./following::*//span")[1].text
#             # contract_type_icon = div_offer.find_element(By.CSS_SELECTOR,"i[name='contract']")
#             # main_contract_div = driver.execute_script("return arguments[0].parentNode;", contract_type_icon)
#             # contract_type = main_contract_div.find_element(By.CSS_SELECTOR,"span").text
#             # published_date = div_offer.find_element(By.CSS_SELECTOR,"time").get_attribute("datetime").split("T")[0]
#             # published_date = datetime.strptime(published_date, '%Y-%m-%d')
#             # published_date = published_date.strftime('%d/%m/%Y')
#             # driver.execute_script("arguments[0].scrollIntoView(true);", div_elements_to_click)
#             # company = div_elements_to_click.find_element(By.CSS_SELECTOR, 'p.card-offer__company').text
#             # contract_type = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[0].text
#             # workplace = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[1].text
#             # published_date = div_elements_to_click.find_elements(By.CSS_SELECTOR, 'ul.important-list > li')[2].text
#             # Scroll into view
#             # actions = ActionChains(driver)
#             # actions.move_to_element(div_elements_to_click).click().perform()
#             #On remote de deux niveau du bouton "Voir offre" pour pouvoir récupérer les informations appropriées
#             # div_offer = driver.execute_script("return arguments[0].parentNode.parentNode;", div_elements_to_click)
#             # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#             # company = company_elem.find_element(By.CSS_SELECTOR, "span").text
#             # company_elem = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='companyName']")
#             # contract_type = div_offer.find_element(By.CSS_SELECTOR, "span[data-cy='contract']").text
#             # workplace = div_offer.find_elements(By.CSS_SELECTOR, "div[data-cy='loc']")[-1].text
#             # published_date = div_offer.find_elements(By.CSS_SELECTOR, "span[data-cy='publishDate']")[-1].text

#             driver.execute_script("arguments[0].click();", div_elements_to_click)

#             WebDriverWait(driver, 3000).until(
#                 EC.presence_of_element_located((By.CSS_SELECTOR, "div[class='media-body']"))
#             )
            
#             # workplace = driver.find_element(By.CSS_SELECTOR,"span[id='lblCity']").text
#             # contract_type_elem = driver.find_element(By.CSS_SELECTOR,"span[id='ltEmploymentType']")
#             # contract_type = contract_type_elem.find_element(By.CSS_SELECTOR,"a").text
            
#             # # driver.execute_script("arguments[0].click();",description_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))
            
#             # WebDriverWait(driver, 3)

#             # try:
#             #     profile_elem = driver.find_element(By.CSS_SELECTOR, "div[data-testid='job-section-experience']")
#             #     profile_html = profile_elem.find_elements(By.CSS_SELECTOR, "div")[-2].get_attribute("innerHTML")
#             #     profile = BeautifulSoup(profile_html, 'html.parser').get_text()
#             # except NoSuchElementException:
#             #     # If profile element is not found, set it to "NULL"
#             #     profile = "NULL"
#             # driver.execute_script("arguments[0].click();",profile_elem.find_elements(By.CSS_SELECTOR,"div")[-1].find_element(By.CSS_SELECTOR,"span"))

#             # description = description_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
#             # profile = profile_elem.find_elements(By.CSS_SELECTOR,"div")[-2].get_attribute("innerHTML")
            
#             #parsing tags
#             # description = BeautifulSoup(description, 'html.parser').get_text()
#             # company_elem = driver.find_element(By.CSS_SELECTOR,"a[href*='/fr/companies/']")
#             # company = company_elem.find_element(By.CSS_SELECTOR,"img").get_attribute("alt")
#             # long_infos = " ".join([description,profile])
#             # print(long_infos)
#             # Wait for the child element to become present in the DOM

#             # # Get the page source after the click
#             # page_source = driver.page_source

#             # Use Beautiful Soup to parse the page source
#             # soup = BeautifulSoup(page_source, 'html.parser')

#             # Example: Retrieve the text of a specific element
#             # desc = soup.select("h4:contains('Descriptif du poste') + p")
#             # descText = ["".join(elem.text) for elem in desc][0]

#             # profile = soup.select("h4:contains('Profil recherché') + p")
#             # profileText = ["".join(elem.text) for elem in profile][0]                  

#             # position = soup.select_one("span[data-cy='jobTitle']").text
#             # position_type = soup.select_one('h4:contains("Secteur d’activité du poste")+span').text
#             # long_infos = soup.select("section")
#             # infos = [item.text.replace("\n"," ").strip() for item in long_infos]
#             # infos = [re.sub(r'\s+', ' ',item) for item in infos]
#             # infos = " ".join([item for item in infos if item != ''])
#             # infos = [item for item in infos if '' not in item]
#             # corpus.append({"source":source,"link":rootLink,"position":position,"position_type":"NULL","company":company,"workplace":workplace,"published_date":published_date,"contract_type":contract_type,"description":long_infos})
#         except Exception as e:
#             # Print the exception for debugging purposes
#             print(f"Error: {e}")

#         finally:
#             # Navigate back to the main page
#             driver.execute_script("window.history.go(-1);")
# finally:
#     driver.quit()

Request failed with status code 404
<!DOCTYPE html>
<html lang="fr" xml:lang="fr" class="no-js">
	<head> 
 <meta name="robots" content="noindex,nofollow"/>
 <meta name="pragma" content="no-cache"/>
		<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
		<meta charset="utf-8">
		<meta http-equiv="X-UA-Compatible" content="IE=edge">
		<meta name="viewport" content="width=device-width, initial-scale=1">

		<!-- The above 3 meta tags *must* come first in the head; any other head content must come *after* these tags -->
		<meta name="description" content="">
		<meta name="author" content="">
		<link rel="icon" href="favicon.ico">

		<title>Erreur 404 | pole-emploi.fr, fusion des sites anpe.fr et assedic.fr</title>
		<link href="/css/pages-autonomes.css" rel="stylesheet">
		<!-- JS -->
		<script src="/js/jquery.min.js"></script>
		<!--[if lt IE 9]>
			<script src="/js/ie8-svg-support.js"></script>
			<script src="/js/html5shiv.min.js"></script>
			<script src="/js/respond.min